In [106]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession 

In [3]:
spark=SparkSession.builder.getOrCreate()

25/03/18 20:20:29 WARN Utils: Your hostname, sherif-Virtual-Machine resolves to a loopback address: 127.0.1.1; using 172.29.81.158 instead (on interface eth0)
25/03/18 20:20:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 20:20:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/18 20:20:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
sc=spark.sparkContext

In [32]:
data=([('Sherif',23),('Shahoda',22),('ahmed',25),('reem',22),('Sherif',22),('Shahoda',20),
     ('ahmed',18),('sherif',30)])
col=['name','age']

In [33]:
rdd0=sc.parallelize(data)

In [34]:
rdd0.collect()

[('Sherif', 23),
 ('Shahoda', 22),
 ('ahmed', 25),
 ('reem', 22),
 ('Sherif', 22),
 ('Shahoda', 20),
 ('ahmed', 18),
 ('sherif', 30)]

### data Frame .create

In [35]:
df=spark.createDataFrame(data,col)

In [36]:
df.show()

+-------+---+
|   name|age|
+-------+---+
| Sherif| 23|
|Shahoda| 22|
|  ahmed| 25|
|   reem| 22|
| Sherif| 22|
|Shahoda| 20|
|  ahmed| 18|
| sherif| 30|
+-------+---+



In [37]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [39]:
avg_age=df.groupby('name').avg('age')

In [40]:
avg_age.show()

+-------+--------+
|   name|avg(age)|
+-------+--------+
| Sherif|    22.5|
|Shahoda|    21.0|
|  ahmed|    21.5|
|   reem|    22.0|
| sherif|    30.0|
+-------+--------+



In [41]:
avg_age.rdd.getNumPartitions()

1

### use sql function and agg


In [42]:
from pyspark.sql.functions import avg

In [43]:
df_avg_age=df.groupby('name').agg(avg('age').alias('Avg_age'))

In [44]:
df_avg_age.show()

+-------+-------+
|   name|Avg_age|
+-------+-------+
| Sherif|   22.5|
|Shahoda|   21.0|
|  ahmed|   21.5|
|   reem|   22.0|
| sherif|   30.0|
+-------+-------+



In [45]:
df_avg_age.printSchema()

root
 |-- name: string (nullable = true)
 |-- Avg_age: double (nullable = true)



### read json file

In [49]:
data=spark.read.json('people.json')

In [50]:
data.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [51]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [52]:
df['name']

Column<'name'>

### sql operations


In [53]:
df_name=df.select('name')

In [54]:
df_name

DataFrame[name: string]

In [55]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [56]:
df.show()

+-------+---+
|   name|age|
+-------+---+
| Sherif| 23|
|Shahoda| 22|
|  ahmed| 25|
|   reem| 22|
| Sherif| 22|
|Shahoda| 20|
|  ahmed| 18|
| sherif| 30|
+-------+---+



#### where cluase

In [59]:
df_age=df.select('age')
df_age.show()

+---+
|age|
+---+
| 23|
| 22|
| 25|
| 22|
| 22|
| 20|
| 18|
| 30|
+---+



In [62]:
from pyspark.sql.functions import col

In [66]:
adult=df_age.where(col('age')>20)
adult.show()

+---+
|age|
+---+
| 23|
| 22|
| 25|
| 22|
| 22|
| 30|
+---+



In [67]:
from pyspark.sql.functions import *
import pyspark.sql.functions as fn

In [68]:
df.show()

+-------+---+
|   name|age|
+-------+---+
| Sherif| 23|
|Shahoda| 22|
|  ahmed| 25|
|   reem| 22|
| Sherif| 22|
|Shahoda| 20|
|  ahmed| 18|
| sherif| 30|
+-------+---+



In [70]:
df.groupby('name').avg('age').show()

+-------+--------+
|   name|avg(age)|
+-------+--------+
| Sherif|    22.5|
|Shahoda|    21.0|
|  ahmed|    21.5|
|   reem|    22.0|
| sherif|    30.0|
+-------+--------+



#### using aggeragiation fun and use alias

In [72]:
df.groupby('name').agg(fn.avg('age').alias('Average age')).show()

+-------+-----------+
|   name|Average age|
+-------+-----------+
| Sherif|       22.5|
|Shahoda|       21.0|
|  ahmed|       21.5|
|   reem|       22.0|
| sherif|       30.0|
+-------+-----------+



### we use agg function to integrate many agregiation function

In [74]:
df.groupby('name').agg(fn.avg('age').alias('Average age'),
                       sum('age').alias('sum_all_ages')).show()

+-------+-----------+------------+
|   name|Average age|sum_all_ages|
+-------+-----------+------------+
| Sherif|       22.5|          45|
|Shahoda|       21.0|          42|
|  ahmed|       21.5|          43|
|   reem|       22.0|          22|
| sherif|       30.0|          30|
+-------+-----------+------------+



### save dataframe in cache

In [75]:
df.persist()

DataFrame[name: string, age: bigint]

``` now data is more faster in access ```

#### sorting

In [79]:
sorting_df=df.sort('age')
sorting_df.show()

+-------+---+
|   name|age|
+-------+---+
|  ahmed| 18|
|Shahoda| 20|
|Shahoda| 22|
| Sherif| 22|
|   reem| 22|
| Sherif| 23|
|  ahmed| 25|
| sherif| 30|
+-------+---+



### create specific schema

In [88]:
from pyspark.sql.types import *

In [93]:
my_schema = StructType([
    StructField('FName', StringType(), False),  # First Name (String, Not Nullable)
    StructField('LName', StringType(), True),    # Last Name (String, Nullable)
    StructField('Age', IntegerType(), False),   # Age (Integer, Not Nullable)
    StructField('Address', StringType(), False),# Address (String, Not Nullable)
    StructField('Gender', StringType(), False)  # Gender (String, Not Nullable)
])

#### create fake data

In [94]:
#!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 6.1 MB/s eta 0:00:006.9 MB/s eta 0:00:01


In [103]:
from faker import Faker
fake=Faker()

data=[]

for _ in range(50):  #create 50 sample of th data
    first_name=fake.first_name()
    last_name=fake.last_name()
    age=fake.random_int(min=18,max=65) ## create random age between 18 and 65
    address=fake.address().replace('\n',', ') ## replace new line with comma
    gender=fake.random_element(elements=('M','F'))

    data.append((first_name,last_name,age,address,gender))

In [104]:
ex_data=spark.createDataFrame(data,my_schema)

In [107]:
ex_data.show()

+---------+--------+---+--------------------+------+
|    FName|   LName|Age|             Address|Gender|
+---------+--------+---+--------------------+------+
|    Bryan|Cisneros| 34|5589 Long Ways, W...|     M|
|    Karen|  Hanson| 46|6654 Marc Ridge, ...|     M|
|     Mary| Mcguire| 35|98738 Wilson Esta...|     M|
|   Johnny|   Hayes| 49|014 Hayden Passag...|     F|
|Katherine|  Morris| 56|46140 Michelle St...|     F|
| Samantha|     Kim| 52|263 Harold Grove ...|     F|
|   Teresa|  Vargas| 25|26365 Coleman Par...|     F|
|  Anthony|   Meyer| 21|370 Petty Grove, ...|     F|
| Margaret|  Austin| 62|06839 Kathryn Tra...|     F|
|    Jorge|    Cruz| 29|32825 Powell Rive...|     F|
|    Peter|   Bowen| 63|389 Carmen Keys, ...|     F|
|    Tammy|  Austin| 57|96681 Tammy Rue, ...|     M|
|    Emily|  Butler| 41|0980 Stephanie Hi...|     M|
|    Craig|   Ramos| 26|0305 Matthew Exte...|     M|
|   Laurie|Williams| 53|01156 Nicole Cent...|     F|
| Brittany| Marquez| 60|249 David View, T...| 

In [129]:
ex_data.printSchema()

root
 |-- FName: string (nullable = false)
 |-- LName: string (nullable = true)
 |-- Age: integer (nullable = false)
 |-- Address: string (nullable = false)
 |-- Gender: string (nullable = false)



In [130]:
ex_data.cache

<bound method DataFrame.cache of DataFrame[FName: string, LName: string, Age: int, Address: string, Gender: string]>

In [133]:
dfx1=ex_data.select(col('Age')*10)
dfx1.show(5)

+----------+
|(Age * 10)|
+----------+
|       340|
|       460|
|       350|
|       490|
|       560|
+----------+
only showing top 5 rows



In [138]:
## or 
dfx2=ex_data.select(expr('Age *10').alias('_10age'))
dfx2.show(5)

+------+
|_10age|
+------+
|   340|
|   460|
|   350|
|   490|
|   560|
+------+
only showing top 5 rows



### create new columns

In [143]:
dfx3=ex_data.withColumn('adult_or_not',col('Age')<30)
dfx3.show(10)

+---------+--------+---+--------------------+------+------------+
|    FName|   LName|Age|             Address|Gender|adult_or_not|
+---------+--------+---+--------------------+------+------------+
|    Bryan|Cisneros| 34|5589 Long Ways, W...|     M|       false|
|    Karen|  Hanson| 46|6654 Marc Ridge, ...|     M|       false|
|     Mary| Mcguire| 35|98738 Wilson Esta...|     M|       false|
|   Johnny|   Hayes| 49|014 Hayden Passag...|     F|       false|
|Katherine|  Morris| 56|46140 Michelle St...|     F|       false|
| Samantha|     Kim| 52|263 Harold Grove ...|     F|       false|
|   Teresa|  Vargas| 25|26365 Coleman Par...|     F|        true|
|  Anthony|   Meyer| 21|370 Petty Grove, ...|     F|        true|
| Margaret|  Austin| 62|06839 Kathryn Tra...|     F|       false|
|    Jorge|    Cruz| 29|32825 Powell Rive...|     F|        true|
+---------+--------+---+--------------------+------+------------+
only showing top 10 rows



### concate 

In [146]:
dfx4=ex_data.withColumn('Full_name',concat_ws(' ',ex_data.FName,col('LName')))
dfx4.show()

+---------+--------+---+--------------------+------+----------------+
|    FName|   LName|Age|             Address|Gender|       Full_name|
+---------+--------+---+--------------------+------+----------------+
|    Bryan|Cisneros| 34|5589 Long Ways, W...|     M|  Bryan Cisneros|
|    Karen|  Hanson| 46|6654 Marc Ridge, ...|     M|    Karen Hanson|
|     Mary| Mcguire| 35|98738 Wilson Esta...|     M|    Mary Mcguire|
|   Johnny|   Hayes| 49|014 Hayden Passag...|     F|    Johnny Hayes|
|Katherine|  Morris| 56|46140 Michelle St...|     F|Katherine Morris|
| Samantha|     Kim| 52|263 Harold Grove ...|     F|    Samantha Kim|
|   Teresa|  Vargas| 25|26365 Coleman Par...|     F|   Teresa Vargas|
|  Anthony|   Meyer| 21|370 Petty Grove, ...|     F|   Anthony Meyer|
| Margaret|  Austin| 62|06839 Kathryn Tra...|     F| Margaret Austin|
|    Jorge|    Cruz| 29|32825 Powell Rive...|     F|      Jorge Cruz|
|    Peter|   Bowen| 63|389 Carmen Keys, ...|     F|     Peter Bowen|
|    Tammy|  Austin|

In [110]:
ex_data.sort('age').show()

+--------+---------+---+--------------------+------+
|   FName|    LName|Age|             Address|Gender|
+--------+---------+---+--------------------+------+
|    Luis|    Hicks| 19|2929 Katherine Ga...|     M|
| Anthony|    Meyer| 21|370 Petty Grove, ...|     F|
|   Scott|  Stewart| 21|26387 Owens Pike,...|     F|
|  Daniel|     Berg| 23|5076 Barrera Prai...|     F|
|  Edward|      Lee| 24|53445 Taylor Ridg...|     M|
|  Austin|   Kramer| 24|53854 Angela Isla...|     M|
| Whitney|   Howard| 24|12100 John Hills ...|     F|
|  Teresa|   Vargas| 25|26365 Coleman Par...|     F|
|   Scott|     Lamb| 25|Unit 8739 Box 337...|     M|
|Danielle|   Miller| 26|667 Denise Viaduc...|     F|
|   Craig|    Ramos| 26|0305 Matthew Exte...|     M|
|Brittany|      Ray| 28|977 Williams Via,...|     M|
| Richard|  Russell| 28|15663 Adkins Rive...|     F|
|   Jorge|     Cruz| 29|32825 Powell Rive...|     F|
| Raymond|   Murray| 30|44344 Daniel Lane...|     F|
|  Nathan|     Rose| 31|331 George Underp...| 

In [116]:
ex1=ex_data.groupby('Gender').avg()
ex1.show()

+------+------------------+
|Gender|          avg(Age)|
+------+------------------+
|     F| 39.73076923076923|
|     M|37.583333333333336|
+------+------------------+



In [117]:
ex2=ex_data.groupby('Gender').agg(fn.avg('Age').alias('average_age'))
ex2.show()

+------+------------------+
|Gender|       average_age|
+------+------------------+
|     F| 39.73076923076923|
|     M|37.583333333333336|
+------+------------------+



In [118]:
ex3=ex_data.select('Address')
ex4=ex3.where(col('age')<35)
ex4.show()

+--------------------+
|             Address|
+--------------------+
|5589 Long Ways, W...|
|26365 Coleman Par...|
|370 Petty Grove, ...|
|32825 Powell Rive...|
|0305 Matthew Exte...|
|Unit 8739 Box 337...|
|53854 Angela Isla...|
|15663 Adkins Rive...|
|12100 John Hills ...|
|25099 Jacob Forge...|
|7665 Williams Vis...|
|5076 Barrera Prai...|
|331 George Underp...|
|08314 Gill Groves...|
|2929 Katherine Ga...|
|26387 Owens Pike,...|
|53445 Taylor Ridg...|
|1988 Johnson Hill...|
|44344 Daniel Lane...|
|977 Williams Via,...|
+--------------------+
only showing top 20 rows



In [128]:
ex5=ex_data.select('Address').filter(col('Address').like('%Forge%'))
ex5.show()

+--------------------+
|             Address|
+--------------------+
|207 Santana Forge...|
|25099 Jacob Forge...|
+--------------------+

